## Usama Abdul Samad<br>
## 311458<br>
## Final Exercise Implementation Code<br>
## Natural Language Processing 

## Importing Libraries

In [171]:
from bs4 import BeautifulSoup
import os
from os import listdir
import random
from nltk.corpus import wordnet as wn
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from numpy import dot
from numpy.linalg import norm
import gensim.downloader
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category= RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Task No 1

# Importing Semcor 3.0 Dataset Files one by one

Imported Brown 1 File

In [172]:
all_samples = []
data_archive = 'semcor3.0\\brown1\\tagfiles'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)

Imported Brown 2 File

In [173]:
data_archive_2 = 'semcor3.0\\brown2\\tagfiles'
for files_2 in os.listdir(data_archive_2):
    with open(os.path.join(data_archive_2 , files_2)) as fp_2:
        soup_2 = BeautifulSoup(fp_2)
        tags_2 = soup_2.find_all('s')
        all_samples.extend(tags_2)

Imported Brown V File

In [174]:
data_archive_3 = 'semcor3.0\\brownv\\tagfiles'
for files_3 in os.listdir(data_archive_3):
    with open(os.path.join(data_archive_3 , files_3)) as fp_3:
        soup_3 = BeautifulSoup(fp_3)
        tags_3 = soup_3.find_all('s')
        all_samples.extend(tags_3)

Making random selection of 5000 by using random sample taking 5000 sentences from all samples

In [175]:
random_selection = random.sample(all_samples, 5000)

# Implementation of Plain Lesk Algorithm and Most Common Sense for SEMCOR 3.0 DATASET

In [176]:
def tagtostring(sentence):                   #function converting the corpus tags to string
    new_sentence = ""
    for x in sentence.contents:
        if x != '\n':
            new_sentence += x.string
            new_sentence += " "
    return new_sentence  

def overlapcontext(synset, sentence):         #Function for getting the overlapcontext 
    gloss = set(synset.definition().split(" "))
    sentence = set(sentence.split(" "))       #Spliting of input sentence on space
    return len( gloss.intersection(sentence) )#getting the intersection between gloss and sentence

def lesk(word, sentence):                     #Plain lesk algorithm
    bestsense = None
    maxoverlap = 0                            #returns the synset with max overlap in terms of meaning
    for sense in wn.synsets(word):
        overlap = overlapcontext(sense,sentence)
        for h in sense.hyponyms():
            overlap += overlapcontext( h, sentence ) #getting the maxoverlaps
        if overlap > maxoverlap:
                maxoverlap = overlap
                bestsense = sense             #getting the best sense to return 
    return bestsense

def most_common_sence(word):                  #function to get the most common sense
    common_sense = wn.synsets(word)
    return None if len(common_sense) == 0 else common_sense[0]

def map(pos):                                 #Mapping function used to map noun adj verb adv etc
    tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV, 'NNP' :wn.NOUN}
    return tag[pos]

Running the main algorithm for Lesk Implementation

In [177]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i]) #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk(w,sentences)                #applying plain lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [178]:
accuracy = (correct / total_checked)* 100
print("Printing results for LESK ALGORITHM")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing results for LESK ALGORITHM
Accuracy with comparison of our prediction and ground truth: 35.20931841771422 %


Running the main algorithm for Most Common Algorithm

In [179]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = most_common_sence(w)             #applying most common sense
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [180]:
accuracy = (correct / total_checked)* 100
print("Printing results for MOST COMMON SENSE Algorithm")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing results for MOST COMMON SENSE Algorithm
Accuracy with comparison of our prediction and ground truth: 44.98904393956868 %


# Paper Implementation for Semcor 3.0 Dataset

Importing lexeme file from the given embeddings

In [181]:
lexeme = open('lexemes.txt','r')  #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')#Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))#converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Doing the sorting as per the paper, by sorting the words in a sentence by there number of synsets

In [182]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Importing the Gensim Embeddings

In [183]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writting the functions for paper implementation 

In [184]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    defination = defination.strip().split(' ')           #splits the definition on space and strip
    for words in defination:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #empty list array
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.strip().split(' ')             #Split the input sentences on space
    for eachword in sentences:                           #Gets each word in a sentence
        if eachword != words:                            #checks if that word is not equal to the input word
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         #does the sorting based on the number of synsets
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the main algorithm for Paper Implementation

In [185]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [340]:
accuracy = (correct / total_checked)* 100
print("Printing results for Paper Implemention for SEMCOR 3.0 DATASET")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing results for Paper Implemention for SEMCOR 3.0 DATASET
Accuracy with comparison of our prediction and ground truth: 45.709971937108367 %


# Importing Files for Senseval 2 Dataset

Importing Files for Senseval 2 Dataset

In [187]:
all_samples = []
data_archive = 'senseval2.semcor\\wordnet3.0'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)
random_selection = random.sample(all_samples, 200)

# Implementation of Plain Lesk Algorithm and Most Common Sense for SENSEVAL 2 DATASET

In [188]:
def tagtostring(sentence):                   #function converting the corpus tags to string
    new_sentence = ""
    for x in sentence.contents:
        if x != '\n':
            new_sentence += x.string
            new_sentence += " "
    return new_sentence  

def overlapcontext(synset, sentence):         #Function for getting the overlapcontext 
    gloss = set(synset.definition().split(" "))
    sentence = set(sentence.split(" "))       #Spliting of input sentence on space
    return len( gloss.intersection(sentence) )#getting the intersection between gloss and sentence

def lesk(word, sentence):                     #Plain lesk algorithm
    bestsense = None
    maxoverlap = 0                            #returns the synset with max overlap in terms of meaning
    for sense in wn.synsets(word):
        overlap = overlapcontext(sense,sentence)
        for h in sense.hyponyms():
            overlap += overlapcontext( h, sentence ) #getting the maxoverlaps
        if overlap > maxoverlap:
                maxoverlap = overlap
                bestsense = sense             #getting the best sense to return 
    return bestsense

def most_common_sence(word):                  #function to get the most common sense
    common_sense = wn.synsets(word)
    return None if len(common_sense) == 0 else common_sense[0]

def map(pos):                                 #Mapping function used to map noun adj verb adv etc
    tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV, 'NNP' :wn.NOUN}
    return tag[pos]

Running the Main Algorithm for Lesk Implementation

In [189]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk(w,sentences)                #applying plain lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [190]:
accuracy = (correct / total_checked)* 100
print("Printing for Senseval 2 Dataset")
print("Printing results for LESK ALGORITHM")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing for Senseval 2 Dataset
Printing results for LESK ALGORITHM
Accuracy with comparison of our prediction and ground truth: 35.66433566433567 %


Runing the Main Algorithm for Most Common Sense

In [191]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = most_common_sence(w)             #applying most common sense
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [192]:
accuracy = (correct / total_checked)* 100
print("Printing for Senseval 2 Dataset")
print("Printing results for MOST COMMON SENSE Algorithm")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing for Senseval 2 Dataset
Printing results for MOST COMMON SENSE Algorithm
Accuracy with comparison of our prediction and ground truth: 40.05994005994006 %


# Paper Implementation for Senseval 2 Dataset

Importing lexeme file from the given embeddings

In [193]:
lexeme = open('lexemes.txt','r')                      #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')                    #Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))                    #converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Doing the sorting as per the paper, by sorting the words in a sentence by there number of synsets

In [194]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Importing the Gensim Embeddings

In [195]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writing functions for paper implemention

In [196]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    defination = defination.strip().split(' ')           #splits the definition on space and strip
    for words in defination:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #empty list array
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.strip().split(' ')             #Split the input sentences on space
    for eachword in sentences:                           #Gets each word in a sentence
        if eachword != words:                            #checks if that word is not equal to the input word
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         #does the sorting based on the number of synsets
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the algorithm for Paper implementation for Senseval 2 dataset

In [197]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [341]:
accuracy = (correct / total_checked)* 100
print("Printing results for Paper Implemention for SENSVAL 2 DATASET")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%"")

Printing results for Paper Implemention for SENSVAL 2 DATASET
Accuracy with comparison of our prediction and ground truth: 45.01873126873127 %


# Importing files for SENSEVAL 3 DATASET

Importing files for Senseval 3 Dataset

In [211]:
all_samples = []
data_archive = 'senseval3.semcor\\wordnet3.0'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)
random_selection = random.sample(all_samples, 250)

# Implementation of Plain Lesk Algorithm and Most Common Sense for SENSEVAL 3 DATASET

In [212]:
def tagtostring(sentence):                   #function converting the corpus tags to string
    new_sentence = ""
    for x in sentence.contents:
        if x != '\n':
            new_sentence += x.string
            new_sentence += " "
    return new_sentence  

def overlapcontext(synset, sentence):         #Function for getting the overlapcontext 
    gloss = set(synset.definition().split(" "))
    sentence = set(sentence.split(" "))       #Spliting of input sentence on space
    return len( gloss.intersection(sentence) )#getting the intersection between gloss and sentence

def lesk(word, sentence):                     #Plain lesk algorithm
    bestsense = None
    maxoverlap = 0                            #returns the synset with max overlap in terms of meaning
    for sense in wn.synsets(word):
        overlap = overlapcontext(sense,sentence)
        for h in sense.hyponyms():
            overlap += overlapcontext( h, sentence ) #getting the maxoverlaps
        if overlap > maxoverlap:
                maxoverlap = overlap
                bestsense = sense             #getting the best sense to return 
    return bestsense

def most_common_sence(word):                  #function to get the most common sense
    common_sense = wn.synsets(word)
    return None if len(common_sense) == 0 else common_sense[0]

def map(pos):                                 #Mapping function used to map noun adj verb adv etc
    tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV, 'NNP' :wn.NOUN}
    return tag[pos]

Running main algorithm for Lesk Implementation

In [213]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                  #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk(w,sentences)                #applying plain lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [214]:
accuracy = (correct / total_checked)* 100
print("Printing for Senseval 3 Dataset")
print("Printing results for LESK ALGORITHM")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing for Senseval 3 Dataset
Printing results for LESK ALGORITHM
Accuracy with comparison of our prediction and ground truth: 36.08490566037736 %


Running main algorithm for Most Common Sense Algorithm

In [215]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = most_common_sence(w)             #applying most common sense
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [216]:
accuracy = (correct / total_checked)* 100
print("Printing for Senseval 3 Dataset")
print("Printing results for MOST COMMON SENSE Algorithm")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

Printing for Senseval 3 Dataset
Printing results for MOST COMMON SENSE Algorithm
Accuracy with comparison of our prediction and ground truth: 45.04716981132076 %


# Paper Implementation for Senseval 3 Dataset


Importing the Lexeme File from the given embeddings

In [217]:
lexeme = open('lexemes.txt','r')                      #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')                    #Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))                    #converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Doing the sorting as per the paper, by sorting the words in a sentence by there number of synsets

In [218]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Importing the Gensim Embeddings

In [219]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writing Functions for paper implementation


In [220]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    defination = defination.strip().split(' ')           #splits the definition on space and strip
    for words in defination:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #empty list array
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.strip().split(' ')             #Split the input sentences on space
    for eachword in sentences:                           #Gets each word in a sentence
        if eachword != words:                            #checks if that word is not equal to the input word
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         #does the sorting based on the number of synsets
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the main algorithm for paper implementation for Senseval 3 dataset

In [221]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [342]:
accuracy = (correct / total_checked)* 100
print("Printing results for Paper Implemention for SENSVAL 3 DATASET")
print("Accuracy with comparison of our prediction and ground truth: ",accuracy,"%")

Printing results for Paper Implemention for SENSVAL 3 DATASET
Accuracy with comparison of our prediction and ground truth: 46.450943396226417 %


# Applying Extensions on the Paper Implementation

###### 1 : For SEMCOR 3.0 DATASET
###### Experiment with removing stopwords and punctuation from the dictionary glosses,sensedescriptionsandcontextsintheoccurrencesofthewordsbefore measuring the distance

Importing Libraries for stopwords and Punctuations

In [316]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import string

Importing the Semcor 3.0 files

In [317]:
all_samples = []
data_archive = 'semcor3.0\\brown1\\tagfiles'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)
data_archive_2 = 'semcor3.0\\brown2\\tagfiles'
for files_2 in os.listdir(data_archive_2):
    with open(os.path.join(data_archive_2 , files_2)) as fp_2:
        soup_2 = BeautifulSoup(fp_2)
        tags_2 = soup_2.find_all('s')
        all_samples.extend(tags_2)
data_archive_3 = 'semcor3.0\\brownv\\tagfiles'
for files_3 in os.listdir(data_archive_3):
    with open(os.path.join(data_archive_3 , files_3)) as fp_3:
        soup_3 = BeautifulSoup(fp_3)
        tags_3 = soup_3.find_all('s')
        all_samples.extend(tags_3)

Randomly selecting 5000 sentences

In [318]:
random_selection =random.sample(all_samples, 5000)

Importing the lexeme embedding file from the given embedding

In [319]:
lexeme = open('lexemes.txt','r')  #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')#Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))#converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Sorting the words according to there no of synsets

In [320]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Loading the Gensim Embeddings

In [321]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writing functions for the main paper implementation, in this we included stopwords and punctuation removal too

In [322]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    defination = defination.translate(str.maketrans('', '', string.punctuation))   # Removing punctuations
    word_tokens = word_tokenize(defination)              #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for words in filtered_sentence:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #Empty list array to store
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.translate(str.maketrans('', '', string.punctuation)) #Removing punctuations
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    word_tokens = word_tokenize(sentences)               #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for eachword in filtered_sentence:                           
        if eachword != words:                            
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the main paper implementation

In [ ]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [343]:
accuracy = (correct / total_checked)* 100
print("*************************")
print("EXTENSION FOR STOP WORDS ")
print("Dataset: Semcor 3.0")
print("*************************")
print("Printing results for Paper Implemention for Semcor 3.0 DATASET")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

*************************
EXTENSION FOR STOP WORDS 
Dataset: Semcor 3.0
*************************
Printing results for Paper Implemention for Semcor 3.0 DATASET
Accuracy with comparison of our prediction and ground truth: 46.35587028716546 %


###### 1 : For SENSVAL 2
###### Experiment with removing stopwords and punctuation from the dictionary glosses,sensedescriptionsandcontextsintheoccurrencesofthewordsbefore measuring the distance

Importing The senseval 2 files

In [325]:
all_samples = []
data_archive = 'senseval2.semcor\\wordnet3.0'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)
random_selection = random.sample(all_samples, 200)

Reading the lexeme file from the given embeddings

In [326]:
lexeme = open('lexemes.txt','r')                      #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')                    #Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))                    #converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Sorting the words according to there number of synsets

In [327]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Loading the Gensim Embeddings

In [328]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writing functions for the main paper implementation, in this we included stopwords and punctuation removal too

In [329]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    defination = defination.translate(str.maketrans('', '', string.punctuation))   # Removing punctuations
    word_tokens = word_tokenize(defination)              #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for words in filtered_sentence:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #Empty list array to store
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.translate(str.maketrans('', '', string.punctuation)) #Removing punctuations
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    word_tokens = word_tokenize(sentences)               #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for eachword in filtered_sentence:                           
        if eachword != words:                            
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the main paper implementation 

In [330]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [345]:
accuracy = (correct / total_checked)* 100
print("*************************")
print("EXTENSION FOR STOP WORDS ")
print("Dataset: Senseval 2.0")
print("*************************")
print("Printing results for Paper Implemention for Senseval 3.0 DATASET")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

*************************
EXTENSION FOR STOP WORDS 
Dataset: Senseval 2.0
*************************
Printing results for Paper Implemention for Senseval 3.0 DATASET
Accuracy with comparison of our prediction and ground truth: 47.4212698412698 %


###### 1 : For SENSVAL 3
###### Experiment with removing stopwords and punctuation from the dictionary glosses,sensedescriptionsandcontextsintheoccurrencesofthewordsbefore measuring the distance

Importing the Senseval 3 Files

In [332]:
all_samples = []
data_archive = 'senseval3.semcor\\wordnet3.0'
for files in os.listdir(data_archive):
    with open(os.path.join(data_archive , files)) as fp:
        soup = BeautifulSoup(fp)
        tags = soup.find_all('s')
        all_samples.extend(tags)
random_selection = random.sample(all_samples, 250)

Reading the Lexeme file from the given embeddings

In [333]:
lexeme = open('lexemes.txt','r')                      #getting lexeme file from the embeddings
lexeme.readline()
lexeme_data = {}
temp = []
for line in lexeme:
    temp = line.strip().split(' ')                    #Splitting and stripping on space
    key = temp[0]
    value = temp[1:]
    value_ =list(np.float_(value))                    #converting to float and after float we convert it into a list
    lexeme_data[key]=value_

Sorting the words according to there number of synsets

In [334]:
def sorting(sentences):                                #sorting function to sort the words according to there number of synsets
    no_synsets = []
    temp = sentences.strip().split(' ')
    for words in temp:
        count = len(wn.synsets(words))
        no_synsets.append(count)                       #counting the no of synsets for each word and storing in a list
        sort = np.argsort(no_synsets)                  #sorting in a acending order
        sortings = [x for _,x in sorted(zip(sort,temp))]
    return sortings

Loading the Gensim Embeddings

In [335]:
gensim_embeddings = gensim.downloader.load('word2vec-google-news-300')

Writing functions for the main paper implementation, in this we included stopwords and punctuation removal too

In [336]:
def cos_sim(lista,listb):                                #Function to find the cosine similarity between two lists
    cosine_sim = dot(lista, listb)/(norm(lista)*norm(listb))
    return cosine_sim

def gloss_embeddings(inputs):                            #function to find the gloss embeddings
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    defination = inputs.definition()                     #gets definition of inputs
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    defination = defination.translate(str.maketrans('', '', string.punctuation))   # Removing punctuations
    word_tokens = word_tokenize(defination)              #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for words in filtered_sentence:
        try: 
            gensim = gensim_embeddings.get_vector(words) #apply gensim embeddings on the words in a definition
        except Exception: 
            continue
        vector_array.append(gensim)                      #appending the gensim in vector array
    for v in vector_array:
        addition_vector = addition_vector+v              #adding so that we can do the averaging
    averaging = np.array(addition_vector)/len(vector_array) #averaging by addition and length of the vector array
    return averaging
        
def context_embedding(words,sentences):                  #function to get the context embedding
    lis = []                                             #Empty list array to store
    addition_vector = np.zeros(shape=(300,))             #addition vector array
    vector_array = []
    sentences = sentences.translate(str.maketrans('', '', string.punctuation)) #Removing punctuations
    stop_words = set(stopwords.words('english'))         #Importing english stopwards
    word_tokens = word_tokenize(sentences)               #tokenizing the definations
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #checking for stopwords and removing them
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)                  #filtering out the stopwords
    for eachword in filtered_sentence:                           
        if eachword != words:                            
            lis.append(eachword)
    lis = sorting(" ".join(lis))                         
    for word in lis:
        try: 
            gensim = gensim_embeddings.get_vector(word)  #Applying gensim embeddings
        except Exception: 
            continue
        vector_array.append(gensim)
    for v in vector_array:
        addition_vector = addition_vector+v              #adding for averaging
    averaging =  np.array(addition_vector)/len(vector_array)
    return averaging                                     #returning the average
            
def lesk_paper(word, sentence):                          #Distributional lesk 
    bestsense = None
    max_score = 0                                        #returns the synset with max score
    for sense in wn.synsets(word):
        string_syn = sense.name().split(".")             
        lexeme_key = "{}-wn-2.1-{}-{}".format(string_syn[0],sense.offset(),string_syn[1])
        score = cos_sim(gloss_embeddings(sense),context_embedding(word,sentence)) #calculates the score as per the paper formula
        if lexeme_key in lexeme_data.keys():             #as we have now gloss and context and also lexeme key
            score = score + cos_sim(lexeme_data[lexeme_key],context_embedding(word,sentence))
        if score > max_score:                            #checks if the score is greater
                max_score = score
                bestsense = sense
    return bestsense                                     #returns the best sense based on the score

Running the main paper implementation

In [337]:
correct = 0
total_checked = 0
for i in range(len(random_selection)):
    sentences = tagtostring(random_selection[i])                   #converting tags to strings
    words = sentences.split(" ")
    for w in words:
        try:
            tag_w = random_selection[i].find("wf",text=w)
            if len(wn.synsets(w)) > 1 and tag_w.has_attr("lemma"): #condition if the synsets are greater 
                                                                   #than one and tags has lemma inside them.
                lesk_prediction = lesk_paper(w,sentences)          #applying distributional lesk algorithm
                tag_ = random_selection[i].find("wf",text=w)
                new_tag = '{}.{}.{:02d}'.format(tag_["lemma"], map(tag_["pos"]), int(tag_["wnsn"])) #mapping the pos tag
                new_synset = wn.synset(new_tag)
                if new_synset is not None and lesk_prediction is not None and  new_synset == lesk_prediction:
                    correct +=1
                total_checked +=1
        except Exception:
            continue

In [346]:
accuracy = (correct / total_checked)* 100
print("*************************")
print("EXTENSION FOR STOP WORDS ")
print("Dataset: Senseval 3.0")
print("*************************")
print("Printing results for Paper Implemention for Senseval 3.0 DATASET")
print("Accuracy with comparison of our prediction and ground truth:",accuracy,"%")

*************************
EXTENSION FOR STOP WORDS 
Dataset: Senseval 3.0
*************************
Printing results for Paper Implemention for Senseval 3.0 DATASET
Accuracy with comparison of our prediction and ground truth: 47.004982290436838 %
